In [1]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import plotly.graph_objects as go

In [10]:
# Load the CSV file
data_file = r'C:\Users\nwang\cleaned_target+wf.csv'
df = pd.read_csv(data_file)
# Preprocess the Price column to normalize values
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle cases like $29.99/lb
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  # Handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Grocery Price Comparison"

# Initialize a dictionary to store accumulated prices
accumulated_prices = {"Target": 0, "Wholefoods": 0}

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=[{"label": cat, "value": cat} for cat in df["Category"].unique()],
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
    
    html.Div(id="price-comparison", style={'margin-bottom': '20px'}),
    
    html.Div([
        dcc.Graph(id="price-comparison-chart")
    ])
])

In [11]:
# Define callback to update product results
@app.callback(
    Output("product-results", "children"),
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def search_products(n_clicks, category, product_name):
    if not category or not product_name:
        return "Please select a category and enter a product name."
    
    # Filter data
    filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found."
    
    # Sort by price-to-quantity ratio
    # Sort by price (now numeric)
    sorted_target = filtered_df[filtered_df["Store"] == "Target"].sort_values("Price")
    sorted_wholefoods = filtered_df[filtered_df["Store"] == "Wholefoods"].sort_values("Price")

    
    # Generate results
    target_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']} per unit") for _, row in sorted_target.iterrows()])
    wholefoods_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']} per unit") for _, row in sorted_wholefoods.iterrows()])
    
    return html.Div([
        html.H3("Target:"),
        target_list,
        html.H3("Wholefoods:"),
        wholefoods_list
    ])


In [12]:
# Define callback to update total prices and comparison
@app.callback(
    [Output("price-comparison", "children"),
     Output("price-comparison-chart", "figure")],
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def update_total_prices(n_clicks, category, product_name):
    if not category or not product_name:
        return "", go.Figure()
    
    # Filter data
    filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found.", go.Figure()
    
    # Accumulate prices for the first item in each store (assuming sorted by lowest price)
    target_price = filtered_df[filtered_df["Store"] == "Target"]["Price"].min()
    wholefoods_price = filtered_df[filtered_df["Store"] == "Wholefoods"]["Price"].min()
    
    if pd.notna(target_price):
        accumulated_prices["Target"] += target_price
    if pd.notna(wholefoods_price):
        accumulated_prices["Wholefoods"] += wholefoods_price
    
    # Create comparison chart
    fig = go.Figure(data=[
        go.Bar(name="Target", x=["Total Cost"], y=[accumulated_prices["Target"]]),
        go.Bar(name="Wholefoods", x=["Total Cost"], y=[accumulated_prices["Wholefoods"]])
    ])
    fig.update_layout(title="Total Price Comparison", barmode="group")
    
    # Display total prices
    comparison_text = f"Total Price: Target - ${accumulated_prices['Target']:.2f}, Wholefoods - ${accumulated_prices['Wholefoods']:.2f}"
    return comparison_text, fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[12], line 25, in update_total_prices(
    n_clicks=1,
    category='Fresh Produce',
    product_name='strawberries'
)
     22 wholefoods_price = filtered_df[filtered_df["Store"] == "Wholefoods"]["Price"].min()
     24 if pd.notna(target_price):
---> 25     accumulated_prices["Target"] += target_price
        target_price = '$7.39'
        accumulated_prices["Target"] = 0
        accumulated_prices = {'Target': 0, 'Wholefoods': 0}
     26 if pd.notna(wholefoods_price):
     27     accumulated_prices["Wholefoods"] += wholefoods_price

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[12], line 25, in update_total_prices(
    n_clicks=3,
    category='Fresh Pro

In [6]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
data_file = r'C:\Users\nwang\cleaned_target+wf.csv'
df = pd.read_csv(data_file)

# Preprocess the Price column to normalize values
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle cases like $29.99/lb
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  # Handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Grocery Price Comparison"

# Initialize a dictionary to store accumulated prices
accumulated_prices = {"Target": 0, "Whole Foods": 0}

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=[{"label": cat, "value": cat} for cat in df["Category"].unique()],
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
    
    html.Div(id="price-comparison", style={'margin-bottom': '20px'}),
    
    html.Div([
        dcc.Graph(id="price-comparison-chart")
    ])
])

# Define callback to update product results
@app.callback(
    Output("product-results", "children"),
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def search_products(n_clicks, category, product_name):
    if not category or not product_name:
        return "Please select a category and enter a product name."
    
    # Filter data
    filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found."
    
    # Ensure valid prices
    filtered_df = filtered_df.dropna(subset=["Price"])
    
    # Sort by price
    sorted_target = filtered_df[filtered_df["Store"] == "Target"].sort_values("Price")
    sorted_wholefoods = filtered_df[filtered_df["Store"] == "Whole Foods"].sort_values("Price")
    
    # Generate results
    target_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in sorted_target.iterrows()])
    wholefoods_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in sorted_wholefoods.iterrows()])
    
    return html.Div([
        html.H3("Target:"),
        target_list,
        html.H3("Whole Foods:"),
        wholefoods_list
    ])

# Define callback to update total prices and comparison
@app.callback(
    [Output("price-comparison", "children"),
     Output("price-comparison-chart", "figure")],
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def update_total_prices(n_clicks, category, product_name):
    if not category or not product_name:
        return "", go.Figure()
    
    # Filter data
    filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found.", go.Figure()
    
    # Ensure valid prices for comparison
    filtered_df = filtered_df.dropna(subset=["Price"])
    
    # Accumulate prices for the first item in each store (assuming sorted by lowest price)
    target_price = filtered_df[filtered_df["Store"] == "Target"]["Price"].min()
    wholefoods_price = filtered_df[filtered_df["Store"] == "Whole Foods"]["Price"].min()
    
    if pd.notna(target_price):
        accumulated_prices["Target"] += target_price
    if pd.notna(wholefoods_price):
        accumulated_prices["Whole Foods"] += wholefoods_price
    
    # Create comparison chart
    fig = go.Figure(data=[
        go.Bar(name="Target", x=["Total Cost"], y=[accumulated_prices["Target"]]),
        go.Bar(name="Whole Foods", x=["Total Cost"], y=[accumulated_prices["Whole Foods"]])
    ])
    fig.update_layout(title="Total Price Comparison", barmode="group")
    
    # Display total prices
    comparison_text = f"Total Price: Target - ${accumulated_prices['Target']:.2f}, Whole Foods - ${accumulated_prices['Whole Foods']:.2f}"
    return comparison_text, fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [8]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd

# Load the CSV file
data_file = r'C:\Users\nwang\data_16b.csv'
df = pd.read_csv(data_file)

# Preprocess the Price column to normalize values
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle cases like $29.99/lb
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  # Handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Grocery Price Comparison"

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=[{"label": cat, "value": cat} for cat in df["Category"].unique()],
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
])

# Define callback to update product results
@app.callback(
    Output("product-results", "children"),
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def search_products(n_clicks, category, product_name):
    if not category or not product_name:
        return "Please select a category and enter a product name."
    
    # Filter data
    filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found."
    
    # Ensure valid prices
    filtered_df = filtered_df.dropna(subset=["Price"])
    
    # Sort by price
    sorted_target = filtered_df[filtered_df["Store"] == "Target"].sort_values("Price")
    sorted_wholefoods = filtered_df[filtered_df["Store"] == "Whole Foods"].sort_values("Price")
    
    # Generate results
    target_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in sorted_target.iterrows()])
    wholefoods_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in sorted_wholefoods.iterrows()])
    
    return html.Div([
        html.H3("Target:"),
        target_list,
        html.H3("Whole Foods:"),
        wholefoods_list
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [3]:
import dash
from dash import dcc, html, Input, Output, State
import pandas as pd

# Load the CSV file
data_file = r'C:\Users\nwang\data_16b.csv'
df = pd.read_csv(data_file)

# Preprocess the Price column to normalize values
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle cases like $29.99/lb
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  # Handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Grocery Price Comparison"

# Add "All" to the dropdown options
category_options = [{"label": "All", "value": "All"}] + [
    {"label": cat, "value": cat} for cat in df["Category"].unique()
]

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=category_options,
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
])

# Define callback to update product results
@app.callback(
    Output("product-results", "children"),
    Input("search-button", "n_clicks"),
    State("category-dropdown", "value"),
    State("product-search", "value"),
    prevent_initial_call=True
)
def search_products(n_clicks, category, product_name):
    if not product_name:
        return "Please enter a product name."
    
    # Filter data based on category and product name
    if category == "All" or category is None:
        filtered_df = df[df["Name"].str.contains(product_name, case=False)]
    else:
        filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found."
    
    # Ensure valid prices
    filtered_df = filtered_df.dropna(subset=["Price"])
    
    # Sort by price
    sorted_target = filtered_df[filtered_df["Store"] == "Target"].sort_values("Price")
    sorted_wholefoods = filtered_df[filtered_df["Store"] == "Whole Foods"].sort_values("Price")
    
    # Generate results
    target_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in sorted_target.iterrows()])
    wholefoods_list = html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in sorted_wholefoods.iterrows()])
    
    return html.Div([
        html.H3("Target:"),
        target_list,
        html.H3("Whole Foods:"),
        wholefoods_list
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [12]:
import dash
from dash import dcc, html, Input, Output, State, ctx
import pandas as pd

# Load the CSV file
data_file = r'C:\Users\nwang\data_16b.csv'
df = pd.read_csv(data_file)

# Preprocess the Price column to normalize values
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle cases like $29.99/lb
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  # Handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

# Initialize Dash app with exception suppression
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.title = "Grocery Price Comparison"

# Add "All" to the dropdown options
category_options = [{"label": "All", "value": "All"}] + [
    {"label": cat, "value": cat} for cat in df["Category"].unique()
]

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=category_options,
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Button("More from Target", id="more-target-button", n_clicks=0, style={'display': 'none'}),
        html.Button("More from Whole Foods", id="more-wholefoods-button", n_clicks=0, style={'display': 'none'}),
    ]),
])

# Initialize store limits
store_limits = {"Target": 5, "Whole Foods": 5}

# Callback to search and display products
@app.callback(
    [Output("product-results", "children"),
     Output("more-target-button", "style"),
     Output("more-wholefoods-button", "style")],
    [Input("search-button", "n_clicks"),
     Input("more-target-button", "n_clicks"),
     Input("more-wholefoods-button", "n_clicks")],
    [State("category-dropdown", "value"),
     State("product-search", "value")],
    prevent_initial_call=True
)
def search_products(search_clicks, more_target_clicks, more_wholefoods_clicks, category, product_name):
    # Determine which button was clicked
    triggered_id = ctx.triggered_id
    
    # Reset store limits on search
    if triggered_id == "search-button":
        store_limits["Target"] = 5
        store_limits["Whole Foods"] = 5
    
    # Increment store limits on "More" button clicks
    if triggered_id == "more-target-button":
        store_limits["Target"] += 5
    elif triggered_id == "more-wholefoods-button":
        store_limits["Whole Foods"] += 5
    
    # Validate inputs
    if not product_name:
        return "Please enter a product name.", {"display": "none"}, {"display": "none"}
    
    # Filter data based on category and product name
    if category == "All" or category is None:
        filtered_df = df[df["Name"].str.contains(product_name, case=False)]
    else:
        filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False))]
    
    if filtered_df.empty:
        return "No products found.", {"display": "none"}, {"display": "none"}
    
    # Ensure valid prices
    filtered_df = filtered_df.dropna(subset=["Price"])
    
    # Sort by price
    sorted_target = filtered_df[filtered_df["Store"] == "Target"].sort_values("Price")
    sorted_wholefoods = filtered_df[filtered_df["Store"] == "Whole Foods"].sort_values("Price")
    
    # Limit the number of displayed items
    limited_target = sorted_target.head(store_limits["Target"])
    limited_wholefoods = sorted_wholefoods.head(store_limits["Whole Foods"])
    
    # Generate results
    target_list = html.Div([
        html.H3("Target:"),
        html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in limited_target.iterrows()]),
    ], style={'margin-bottom': '20px'})

    wholefoods_list = html.Div([
        html.H3("Whole Foods:"),
        html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in limited_wholefoods.iterrows()]),
    ], style={'margin-bottom': '20px'})
    
    # Determine button visibility
    target_button_style = {"display": "block"} if len(sorted_target) > store_limits["Target"] else {"display": "none"}
    wholefoods_button_style = {"display": "block"} if len(sorted_wholefoods) > store_limits["Whole Foods"] else {"display": "none"}
    
    return html.Div([target_list, wholefoods_list]), target_button_style, wholefoods_button_style

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


In [21]:
import dash
from dash import dcc, html, Input, Output, State, ctx
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
data_file = r'C:\Users\nwang\data_16b.csv'
df = pd.read_csv(data_file)

# Preprocess the Price column to normalize values
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  # Handle cases like $29.99/lb
            price = price.split("/")[0].strip()  # Extract numeric part
        price = price.replace("$", "").strip()  # Remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  # Handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

# Initialize Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.title = "Grocery Price Comparison"

# Add "All" to the dropdown options
category_options = [{"label": "All", "value": "All"}] + [
    {"label": cat, "value": cat} for cat in df["Category"].unique()
]

# Define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=category_options,
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Button("More from Target", id="more-target-button", n_clicks=0, style={'display': 'none'}),
        html.Button("More from Whole Foods", id="more-wholefoods-button", n_clicks=0, style={'display': 'none'}),
    ]),

    dcc.Graph(id="price-bar-graph"),
])

# Initialize store limits
store_limits = {"Total": 10}

# Callback to search and display products
@app.callback(
    [Output("product-results", "children"),
     Output("more-target-button", "style"),
     Output("more-wholefoods-button", "style"),
     Output("price-bar-graph", "figure")],
    [Input("search-button", "n_clicks"),
     Input("more-target-button", "n_clicks")],
    [State("category-dropdown", "value"),
     State("product-search", "value")],
    prevent_initial_call=True
)
def search_products(search_clicks, more_button_clicks, category, product_name):
    # Determine which button was clicked
    triggered_id = ctx.triggered_id
    
    # Reset store limits on search
    if triggered_id == "search-button":
        store_limits["Total"] = 10
    
    # Increment store limits on "More" button clicks
    if triggered_id == "more-target-button":
        store_limits["Total"] += 10
    
    # Validate inputs
    if not product_name:
        return "Please enter a product name.", {"display": "none"}, {"display": "none"}, go.Figure()
    
    # Filter data based on category and product name
    if category == "All" or category is None:
        filtered_df = df[df["Name"].str.contains(product_name, case=False, na=False)]
    else:
        filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False, na=False))]
    
    if filtered_df.empty:
        return "No products found.", {"display": "none"}, {"display": "none"}, go.Figure()
    
    # Ensure valid prices
    filtered_df = filtered_df.dropna(subset=["Price"])
    
    # Sort all products by price and apply limit
    sorted_df = filtered_df.sort_values("Price").head(store_limits["Total"])
    
    # Generate results
    product_list = html.Div([
        html.Ul([html.Li(f"{row['Name']} - {row['Store']} - ${row['Price']:.2f} per unit") for _, row in sorted_df.iterrows()]),
    ], style={'margin-bottom': '20px'})
    
    # Determine button visibility
    more_button_style = {"display": "block"} if len(filtered_df) > store_limits["Total"] else {"display": "none"}
    
    # Create grouped bar chart
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=list(range(len(sorted_df))),
        y=sorted_df["Price"],
        name="Price",
        marker_color=sorted_df["Store"].map({"Target": "blue", "Whole Foods": "green"}),
        hovertext=sorted_df["Name"] + " (" + sorted_df["Store"] + ")",
        hoverinfo="text+y"
    ))
    fig.update_layout(
        title="Price Comparison Between Stores",
        xaxis_title="Product Index",
        yaxis_title="Price ($)",
        legend_title="Store",
        showlegend=False
    )
    
    return product_list, more_button_style, more_button_style, fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)



In [28]:
import dash
from dash import dcc, html, Input, Output, State, ctx
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file
data_file = r'C:\Users\nwang\data_16b.csv'
df = pd.read_csv(data_file)

#preprocess to clean data
def clean_price(row):
    price = row["Price"]
    if isinstance(price, str):
        if "/" in price:  #handle $3.00/lb cases
            price = price.split("/")[0].strip()  #extract numeric part
        price = price.replace("$", "").strip()  #remove dollar sign
    try:
        return float(price)
    except ValueError:
        return None  #handle invalid prices gracefully

df["Price"] = df.apply(clean_price, axis=1)

#initialize Dash app
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.title = "Grocery Price Comparison"

#add "All" to the dropdown options
category_options = [{"label": "All", "value": "All"}] + [
    {"label": cat, "value": cat} for cat in df["Category"].unique()
]

#define the layout
app.layout = html.Div([
    html.H1("Grocery Price Comparison", style={'text-align': 'center'}),
    
    html.Div([
        html.Label("Choose a Category:"),
        dcc.Dropdown(
            id="category-dropdown",
            options=category_options,
            placeholder="Select a category"
        ),
    ], style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Label("Search for a Product:"),
        dcc.Input(id="product-search", type="text", placeholder="Enter product name..."),
        html.Button("Search", id="search-button", n_clicks=0),
    ], style={'margin-bottom': '20px'}),
    
    html.Div(id="product-results", style={'margin-bottom': '20px'}),
    
    html.Div([
        html.Button("More from Target", id="more-target-button", n_clicks=0, style={'display': 'none'}),
        html.Button("More from Whole Foods", id="more-wholefoods-button", n_clicks=0, style={'display': 'none'}),
    ]),

    dcc.Graph(id="price-bar-graph"),
])

#initialize store limits
store_limits = {"Target": 5, "Whole Foods": 5}

#callback to search and display products
@app.callback(
    [Output("product-results", "children"),
     Output("more-target-button", "style"),
     Output("more-wholefoods-button", "style"),
     Output("price-bar-graph", "figure")],
    [Input("search-button", "n_clicks"),
     Input("more-target-button", "n_clicks"),
     Input("more-wholefoods-button", "n_clicks")],
    [State("category-dropdown", "value"),
     State("product-search", "value")],
    prevent_initial_call=True
)
def search_products(search_clicks, more_target_clicks, more_wholefoods_clicks, category, product_name):
    #determine which button was clicked
    triggered_id = ctx.triggered_id
    
    #reset store limits and graph on search
    if triggered_id == "search-button":
        store_limits["Target"] = 5
        store_limits["Whole Foods"] = 5
    
    #increment store limits on "More" button clicks
    if triggered_id == "more-target-button":
        store_limits["Target"] += 5
    elif triggered_id == "more-wholefoods-button":
        store_limits["Whole Foods"] += 5
    
    #validate inputs
    if not product_name:
        return "Please enter a product name.", {"display": "none"}, {"display": "none"}, go.Figure()
    
    #filter data based on category and product name
    if category == "All" or category is None:
        filtered_df = df[df["Name"].str.contains(product_name, case=False, na=False)]
    else:
        filtered_df = df[(df["Category"] == category) & (df["Name"].str.contains(product_name, case=False, na=False))]
    
    if filtered_df.empty:
        return "No products found.", {"display": "none"}, {"display": "none"}, go.Figure()
    
    #ensure valid prices
    filtered_df = filtered_df.dropna(subset=["Price"])
    
    #sort by price
    sorted_target = filtered_df[filtered_df["Store"] == "Target"].sort_values("Price")
    sorted_wholefoods = filtered_df[filtered_df["Store"] == "Whole Foods"].sort_values("Price")
    
    #limit the number of displayed items
    limited_target = sorted_target.head(store_limits["Target"])
    limited_wholefoods = sorted_wholefoods.head(store_limits["Whole Foods"])
    
    #generate results
    target_list = html.Div([
        html.H3("Target:"),
        html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in limited_target.iterrows()]),
    ], style={'margin-bottom': '20px'})

    wholefoods_list = html.Div([
        html.H3("Whole Foods:"),
        html.Ul([html.Li(f"{row['Name']} - ${row['Price']:.2f} per unit") for _, row in limited_wholefoods.iterrows()]),
    ], style={'margin-bottom': '20px'})
    
    #determine button visibility
    target_button_style = {"display": "block"} if len(sorted_target) > store_limits["Target"] else {"display": "none"}
    wholefoods_button_style = {"display": "block"} if len(sorted_wholefoods) > store_limits["Whole Foods"] else {"display": "none"}
    
    #create grouped bar chart
    fig = go.Figure()
    if not limited_target.empty:
        fig.add_trace(go.Bar(
            x=list(range(len(limited_target))),
            y=limited_target["Price"],
            name="Target",
            marker_color="#8B0000",
            hovertext=limited_target.apply(
                lambda row: f"{row['Name']} (${row['Price']:.2f})", axis=1
            ),
            hoverinfo="text+y"
        ))
    if not limited_wholefoods.empty:
        fig.add_trace(go.Bar(
            x=list(range(len(limited_target), len(limited_target) + len(limited_wholefoods))),
            y=limited_wholefoods["Price"],
            name="Whole Foods",
            marker_color="#BAFFC9",
            hovertext=limited_wholefoods.apply(
                lambda row: f"{row['Name']} (${row['Price']:.2f})", axis=1
            ),
            hoverinfo="text+y"
        ))

    fig.update_layout(
        title="Price Comparison Between Stores",
        xaxis_title="Product Index",
        yaxis_title="Price ($)",
        barmode="group",
        legend_title="Store"
    )
    
    return html.Div([target_list, wholefoods_list]), target_button_style, wholefoods_button_style, fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
